## Schema Design/Critique Notebook
This notebook provides an exploration of the tables created as part of a schema design/critque exercise. In Part 1, we brainstorm a schema design based on some hypothetical use cases. In Part 2, we review and critique a proposed schema for a data source that only consists of one table, and determine if these solutions can be improved. Both parts perform this analysis in the context of database normalization forms (1NF, 2NF, etc.)

In [1]:
# Packages and environment variables
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv, dotenv_values
from IPython.display import display

load_dotenv("../src/.env")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
HOST = os.getenv("HOST")
PORT = os.getenv("PORT")

# Database names
MEDICAL_DB = "medical_center"
CRAIGSLIST_DB = "craigslist"
SOCCER_DB = "soccer"
OUTER_SPACE_DB = "outer_space"
AIR_TRAFFIC_DB = "air_traffic"
MUSIC_DB = "music"

# Helper functions for querying
def run_queries(queries, db_name):
    psql_conn_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{HOST}:{PORT}/{db_name}"
    engine = create_engine(psql_conn_string)
    output_dfs = [pd.read_sql(query, engine) for query in queries]
    engine.dispose()
    return output_dfs

def show_table_examples(table_list, db_name):
    queries = [f"SELECT * FROM {t}" for t in table_list]
    results = [(table_name, df) for table_name, df in zip(table_list, run_queries(queries, db_name))]
    return results

### Part 1: Schema Design
For this section, we explore how the schemas we created in Part 1. The diagrams for these solutions can be viewed in the <tt>docs</tt> section of this repository.

#### Medical Center Use Case
Here, we have generated individual tables pertaining to <tt>patients</tt>, <tt>physicians</tt>, <tt>visits</tt>, <tt>diagnoses</tt>, and <tt>diseases</tt>:

In [2]:
# View medical tables
medical_tables = ["patients","physicians","visits","diagnoses","diseases"]
medical_examples = show_table_examples(medical_tables, MEDICAL_DB)
for e in medical_examples:
    print(e[0])
    display(e[1])

patients


,patient_id,first_name,last_name
0,1,Marty,McFly
1,2,Peter,Parker
2,3,Marty,McFly
3,4,Peter,Parker
4,5,Marty,McFly
5,6,Peter,Parker
6,7,Marty,McFly
7,8,Peter,Parker


physicians


,physician_id,first_name,last_name
0,1,Emmett,Brown
1,2,Otto,Octavius
2,3,Emmett,Brown
3,4,Otto,Octavius
4,5,Emmett,Brown
5,6,Otto,Octavius
6,7,Emmett,Brown
7,8,Otto,Octavius


visits


,encounter_id,encounter_date,physician_id,patient_id
0,1,2024-03-01,1,1
1,2,2024-03-02,2,1
2,3,2024-03-06,2,2
3,4,2024-03-01,1,1
4,5,2024-03-02,2,1
5,6,2024-03-06,2,2
6,7,2024-03-01,1,1
7,8,2024-03-02,2,1
8,9,2024-03-06,2,2
9,10,2024-03-01,1,1


diagnoses


,diagnosis_id,encounter_id,disease_id
0,1,1,1
1,2,2,3
2,3,3,2
3,4,3,3
4,5,1,1
5,6,2,3
6,7,3,2
7,8,3,3
8,9,1,1
9,10,2,3


diseases


,disease_id,icd_version,code_value,code_desc
0,1,9,401.1,Primary Hypertension
1,2,10,I10,Primary Hypertension
2,3,10,W56.01,Bitten by Dolphin
3,4,9,401.1,Primary Hypertension
4,5,10,I10,Primary Hypertension
5,6,10,W56.01,Bitten by Dolphin
6,7,9,401.1,Primary Hypertension
7,8,10,I10,Primary Hypertension
8,9,10,W56.01,Bitten by Dolphin
9,10,9,401.1,Primary Hypertension


In [3]:
# Example medical join
medical_join_query = """select
	a.encounter_id
	, a.encounter_date 
	, b.first_name || ' ' || b.last_name as physician_full_name 
	, c.first_name || ' ' || c.last_name as patient_full_name
	, e.icd_version
	, e.code_value
	, e.code_desc
from 
	visits a 
inner join 
	physicians b
	on a.physician_id = b.physician_id 
inner join 
	patients c
	on a.patient_id = c.patient_id
inner join 
	diagnoses d
	on a.encounter_id = d.encounter_id
inner join 
	diseases e
	on d.disease_id = e.disease_id;"""

medical_join_df = run_queries([medical_join_query], MEDICAL_DB)
medical_join_df[0]

,encounter_id,encounter_date,physician_full_name,patient_full_name,icd_version,code_value,code_desc
0,1,2024-03-01,Emmett Brown,Marty McFly,9,401.1,Primary Hypertension
1,2,2024-03-02,Otto Octavius,Marty McFly,10,W56.01,Bitten by Dolphin
2,3,2024-03-06,Otto Octavius,Peter Parker,10,I10,Primary Hypertension
3,3,2024-03-06,Otto Octavius,Peter Parker,10,W56.01,Bitten by Dolphin
4,1,2024-03-01,Emmett Brown,Marty McFly,9,401.1,Primary Hypertension
5,2,2024-03-02,Otto Octavius,Marty McFly,10,W56.01,Bitten by Dolphin
6,3,2024-03-06,Otto Octavius,Peter Parker,10,I10,Primary Hypertension
7,3,2024-03-06,Otto Octavius,Peter Parker,10,W56.01,Bitten by Dolphin
8,1,2024-03-01,Emmett Brown,Marty McFly,9,401.1,Primary Hypertension
9,2,2024-03-02,Otto Octavius,Marty McFly,10,W56.01,Bitten by Dolphin


#### Craigslist
This schema consists of four tables: <tt>categories</tt>, <tt>individuals</tt>, <tt>posts</tt> and <tt>regions</tt>:

In [4]:
# View Craigslist tables
craigslist_tables = ["categories","individuals","posts","regions"]
craigslist_examples = show_table_examples(craigslist_tables, CRAIGSLIST_DB)
for e in craigslist_examples:
    print(e[0])
    display(e[1])

categories


,category_id,category_name,category_desc
0,1,Technology,Listings for technical expertise
1,2,Deliveries,Listings for delivery services


individuals


,user_id,first_name,last_name,preferred_region_id
0,1,Peter,Griffin,2
1,2,Homer,Simpson,1


posts


,post_id,title,post_text,post_location,user_id,post_region_id,category_id
0,1,Pizza delivery,I need someone to deliver my pizza tonight! Ca...,New York Hotel,2,2,2
1,2,Drop off couch,I need someone to deliver new couch from the s...,123 Western Lane,2,1,2
2,3,Time Travel,Looking for someone to go back in time with me...,123 Quahog Lane,1,2,1


regions


,region_id,region_name,population_num
0,1,West Coast,53000000
1,2,New England,15000000


In [5]:
# Example Craigslist join
craigslist_join_query = """select
	a.post_id
	, b.first_name || ' ' || b.last_name as poster_name
	, c.category_name 
	, c.category_desc
	, a.title
	, a.post_text
	, a.post_location
	, d.region_name as post_region_name
	, e.region_name as preferred_region_name
from
	posts a
inner join 
	individuals b
	on a.user_id = b.user_id
inner join 
	categories c
	on a.category_id = c.category_id
inner join 
	regions d
	on a.post_region_id = d.region_id
inner join 
	regions e
	on b.preferred_region_id = e.region_id;"""

craigslist_join_df = run_queries([craigslist_join_query], CRAIGSLIST_DB)
craigslist_join_df[0]

,post_id,poster_name,category_name,category_desc,title,post_text,post_location,post_region_name,preferred_region_name
0,1,Homer Simpson,Deliveries,Listings for delivery services,Pizza delivery,I need someone to deliver my pizza tonight! Ca...,New York Hotel,New England,West Coast
1,2,Homer Simpson,Deliveries,Listings for delivery services,Drop off couch,I need someone to deliver new couch from the s...,123 Western Lane,West Coast,West Coast
2,3,Peter Griffin,Technology,Listings for technical expertise,Time Travel,Looking for someone to go back in time with me...,123 Quahog Lane,New England,New England


#### Soccer
This schema consists of multiple tables as shown below:

In [6]:
# View Soccer tables
soccer_tables = ["games","goals","matchups","officiators","players","referees","seasons","teams"]
soccer_examples = show_table_examples(soccer_tables, SOCCER_DB)
for e in soccer_examples:
    print(e[0])
    display(e[1])

games


,game_id,game_date,season_id
0,1,2022-08-01,1
1,2,2022-12-01,1
2,3,2023-02-01,1


goals


,goal_id,game_clock_time,game_id,player_id
0,1,5,1,3
1,2,15,1,4
2,3,8,2,5
3,4,10,3,6


matchups


...

,referee_id,first_name,last_name
0,1,Velma,Dinkley
1,2,J. Qunicy,Magoo


seasons


,season_id,season_start_date,season_end_date
0,1,2022-08-01,2023-02-01


teams


,team_id,team_city,team_name
0,1,Dallas,Cattlemen
1,2,Las Vegas,Llamas
2,3,San Francisco,Trolleys


In [7]:
# Example soccer join - standings can be computed from baseline metrics
# described in the project, though this requires joining multiple tables
soccer_join_query = """with team_game_points as
(select
	b.game_id
	, c.team_id
 	, f.team_city
 	, f.team_name
	, sum(case when e.player_id is not null then 1 else 0 end) as goals
from
	seasons a
inner join 
	games b
	on a.season_id = b.season_id
inner join 
	matchups c
	on b.game_id = c.game_id
inner join 
	players d
	on c.team_id = d.team_id
left join 
	goals e
	on d.player_id = e.player_id
 	and b.game_id = e.game_id
inner join 
 	teams f
 	on c.team_id = f.team_id
where 
 	a.season_id = 1
group by 
	b.game_id
	, c.team_id
	, f.team_city
	, f.team_name),
	
differentials as
(select
	game_id
	, team_id
 	, team_city
 	, team_name
	, goals
	, goals - lead(goals) over (partition by game_id order by team_id) as differential
	, lead(team_id) over (partition by game_id order by team_id) as opp_team
	, row_number() over (partition by game_id order by team_id) as rn
from
	team_game_points),
	
win_ids as
(select
	*
	, case
		when differential > 0 then team_id
		when differential < 0 then opp_team
		else null end as winning_team_id
from 
	differentials
where 
	rn = 1),
	
game_counts as
(select 
	team_id
	, team_city
	, team_name
	, count(*) as games_played
from 
	team_game_points
group by 
	team_id 
	, team_city
	, team_name)
	
select 
	a.team_city
	, a.team_name
	, a.games_played
	, sum(case when a.team_id = b.winning_team_id then 1 else 0 end) as wins
	, a.games_played - sum(case when a.team_id = b.winning_team_id then 1 else 0 end) as losses
from
	game_counts a
left join 
	win_ids b
	on a.team_id = b.winning_team_id
group by 
	a.team_city
	, a.team_name
	, a.games_played
order by 
	wins desc;"""

soccer_join_df = run_queries([soccer_join_query], SOCCER_DB)
soccer_join_df[0]

,team_city,team_name,games_played,wins,losses
0,San Francisco,Trolleys,2,2,0
1,Las Vegas,Llamas,2,1,1
2,Dallas,Cattlemen,2,0,2


### Part 2: Schema Critique
In this section, we review schemas that consist of a single table of data to evaluate and implement normalization methods to avoid redundancies and make our data more consise.

#### Outer Space Data
We can see that there are several items that do not coincide with normal forms. For instance, we see a multi-valued attribute (<tt>moons</tt>). This could be split up into a new table. Also, there is a a degree of dependency of where <tt>orbits_around</tt> can imply the <tt>galaxy</tt>. These could be broken up into smaller tables as shown below:

In [8]:
# Initial schema (single table)
outer_space_df = run_queries(["SELECT * from planets"], OUTER_SPACE_DB)[0]
outer_space_df

,id,name,orbital_period_in_years,orbits_around,galaxy,moons
0,1,Earth,1.00,The Sun,Milky Way,[The Moon]
1,2,Mars,1.88,The Sun,Milky Way,"[Phobos, Deimos]"
2,3,Venus,0.62,The Sun,Milky Way,[]
3,4,Neptune,164.80,The Sun,Milky Way,"[Naiad, Thalassa, Despina, Galatea, Larissa, S..."
4,5,Proxima Centauri b,0.03,Proxima Centauri,Milky Way,[]
5,6,Gliese 876 b,0.23,Gliese 876,Milky Way,[]


In [9]:
# View new schema (broken up into normalized tables)
outer_space_tables = ["planets_new","stars","galaxies","moons"]
outer_space_examples = show_table_examples(outer_space_tables, OUTER_SPACE_DB)
for e in outer_space_examples:
    print(e[0])
    display(e[1])

planets_new


,planet_id,planet_name,orbital_period_in_years,star_id
0,1,Earth,1.00,1
1,2,Mars,1.88,1
2,3,Venus,0.62,1
3,4,Neptune,164.80,1
4,5,Proxima Centauri b,0.03,2
5,6,Gliese 876 b,0.23,3


stars


,star_id,star_name,galaxy_id
0,1,The Sun,1
1,2,Proxima Centauri,1
2,3,Gliese 876,1


galaxies


,galaxy_id,galaxy_name
0,1,Milky Way


moons


,moon_id,planet_id,moon_name
0,1,1,The Moon
1,2,2,Phobos
2,3,2,Deimos
3,4,4,Naiad
4,5,4,Thalassa
5,6,4,Despina
6,7,4,Galatea
7,8,4,Larissa
8,9,4,S/2004 N 1
9,10,4,Proteus


#### Air Traffic Data
We can see multiple issues in this case such as the duplication of the candidate key of first/last name due to being matched to multiple flights. We can also observe dependencies in the relationship of city implying country. This can be remediated by splitting the data into multiple tables.

In [10]:
# Initial schema (single table)
air_traffic_df = run_queries(["SELECT * from tickets"], AIR_TRAFFIC_DB)[0]
air_traffic_df

,id,first_name,last_name,seat,departure,arrival,airline,from_city,from_country,to_city,to_country
0,1,Jennifer,Finch,33B,2018-04-08 09:00:00,2018-04-08 12:00:00,United,Washington DC,United States,Seattle,United States
1,2,Thadeus,Gathercoal,8A,2018-12-19 12:45:00,2018-12-19 16:15:00,British Airways,Tokyo,Japan,London,United Kingdom
2,3,Sonja,Pauley,12F,2018-01-02 07:00:00,2018-01-02 08:03:00,Delta,Los Angeles,United States,Las Vegas,United States
3,4,Jennifer,Finch,20A,2018-04-15 16:50:00,2018-04-15 21:00:00,Delta,Seattle,United States,Mexico City,Mexico
4,5,Waneta,Skeleton,23D,2018-08-01 18:30:00,2018-08-01 21:50:00,TUI Fly Belgium,Paris,France,Casablanca,Morocco
5,6,Thadeus,Gathercoal,18C,2018-10-31 01:15:00,2018-10-31 12:55:00,Air China,Dubai,UAE,Beijing,China
6,7,Berkie,Wycliff,9E,2019-02-06 06:00:00,2019-02-06 07:47:00,United,New York,United States,Charlotte,United States
7,8,Alvin,Leathes,1A,2018-12-22 14:42:00,2018-12-22 15:56:00,American Airlines,Cedar Rapids,United States,Chicago,United States
8,9,Berkie,Wycliff,32B,2019-02-06 16:28:00,2019-02-06 19:18:00,American Airlines,Charlotte,United States,New Orleans,United States
9,10,Cory,Squibbes,10D,2019-01-20 19:30:00,2019-01-20 22:45:00,Avianca Brasil,Sao Paolo,Brazil,Santiago,Chile


In [12]:
# View new schema (broken up into normalized tables)
air_traffic_tables = ["airlines","cities","flights","customers","seat_numbers","customers_flights","countries"]
air_traffic_examples = show_table_examples(air_traffic_tables, AIR_TRAFFIC_DB)
for e in air_traffic_examples:
    print(e[0])
    display(e[1])

airlines


,airline_id,airline_name
0,1,United
1,2,British Airways
2,3,Delta
3,4,TUI Fly Belgium
4,5,Air China
5,6,American Airlines
6,7,Avianca Brasil


cities


,city_id,city_name,country_id
0,1,Sao Paolo,1
1,2,Santiago,2
2,3,Beijing,3
3,4,Paris,4
4,5,Tokyo,5
5,6,Mexico City,6
6,7,Casablanca,7
7,8,Dubai,8
8,9,London,9
9,10,Seattle,10


flights


...

,seat_id,seat_number
0,1,10D
1,2,23D
2,3,8A
3,4,18C
4,5,20A
5,6,33B
6,7,12F
7,8,32B
8,9,1A
9,10,9E


customers_flights


,cust_flight_id,cust_id,flight_id,seat_id
0,1,3,1,1
1,2,7,2,2
2,3,6,3,3
3,4,6,4,4
4,5,4,5,5
5,6,4,6,6
6,7,5,7,7
7,8,2,8,8
8,9,1,9,9
9,10,2,10,10


countries


,country_id,country_name
0,1,Brazil
1,2,Chile
2,3,China
3,4,France
4,5,Japan
5,6,Mexico
6,7,Morocco
7,8,UAE
8,9,United Kingdom
9,10,United States


#### Music Data
We can observe that one song can have multiple associated artists and producers which should be separated into their own tables. There is also the potential to repeat record values regarding album title as one album could be related to several songs.

In [13]:
# Initial schema (single table)
music_df = run_queries(["SELECT * from songs"], MUSIC_DB)[0]
music_df

,id,title,duration_in_seconds,release_date,artists,album,producers
0,1,MMMBop,238,1997-04-15,[Hanson],Middle of Nowhere,"[Dust Brothers, Stephen Lironi]"
1,2,Bohemian Rhapsody,355,1975-10-31,[Queen],A Night at the Opera,[Roy Thomas Baker]
2,3,One Sweet Day,282,1995-11-14,"[Mariah Cary, Boyz II Men]",Daydream,[Walter Afanasieff]
3,4,Shallow,216,2018-09-27,"[Lady Gaga, Bradley Cooper]",A Star Is Born,[Benjamin Rice]
4,5,How You Remind Me,223,2001-08-21,[Nickelback],Silver Side Up,[Rick Parashar]
5,6,New York State of Mind,276,2009-10-20,"[Jay Z, Alicia Keys]",The Blueprint 3,[Al Shux]
6,7,Dark Horse,215,2013-12-17,"[Katy Perry, Juicy J]",Prism,"[Max Martin, Cirkut]"
7,8,Moves Like Jagger,201,2011-06-21,"[Maroon 5, Christina Aguilera]",Hands All Over,"[Shellback, Benny Blanco]"
8,9,Complicated,244,2002-05-14,[Avril Lavigne],Let Go,[The Matrix]
9,10,Say My Name,240,1999-11-07,[Destiny's Child],The Writing's on the Wall,[Darkchild]


In [14]:
# View new schema (broken up into normalized tables)
music_tables = ["songs_new","artists","producers","albums","songs_artists","songs_producers"]
music_examples = show_table_examples(music_tables, MUSIC_DB)
for e in music_examples:
    print(e[0])
    display(e[1])

songs_new


,song_id,song_name,duration_in_seconds,release_date,album_id
0,1,MMMBop,238,1997-04-15,1
1,2,Bohemian Rhapsody,355,1975-10-31,2
2,3,One Sweet Day,282,1995-11-14,3
3,4,Shallow,216,2018-09-27,4
4,5,How You Remind Me,223,2001-08-21,5
5,6,New York State of Mind,276,2009-10-20,6
6,7,Dark Horse,215,2013-12-17,7
7,8,Moves Like Jagger,201,2011-06-21,8
8,9,Complicated,244,2002-05-14,9
9,10,Say My Name,240,1999-11-07,10


artists


,artist_id,artist_name
0,1,Hanson
1,2,Queen
2,3,Mariah Cary
3,4,Boyz II Men
4,5,Lady Gaga
5,6,Bradley Cooper
6,7,Nickelback
7,8,Jay Z
8,9,Alicia Keys
9,10,Katy Perry


producers


...

,album_id,album_name
0,1,Middle of Nowhere
1,2,A Night at the Opera
2,3,Daydream
3,4,A Star Is Born
4,5,Silver Side Up
5,6,The Blueprint 3
6,7,Prism
7,8,Hands All Over
8,9,Let Go
9,10,The Writing's on the Wall


songs_artists


,song_artist_id,song_id,artist_id
0,1,1,1
1,2,2,2
2,3,3,3
3,4,3,4
4,5,4,5
5,6,4,6
6,7,5,7
7,8,6,8
8,9,6,9
9,10,7,10


songs_producers


,song_producer_id,song_id,producer_id
0,1,1,1
1,2,1,2
2,3,2,3
3,4,3,4
4,5,4,5
5,6,5,6
6,7,6,7
7,8,7,8
8,9,7,9
9,10,8,10
